In [188]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt*

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

SyntaxError: invalid syntax (299830721.py, line 4)

In [9]:
!curl -O https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 80876  100 80876    0     0  67910      0  0:00:01  0:00:01 --:--:-- 69124


In [190]:
df = pd.read_csv('course_lead_scoring.csv')

In [192]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [194]:
# Count occurrences of each industry
df["industry"].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
Name: count, dtype: int64

In [196]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [198]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [200]:
df['lead_source'].unique()

array(['paid_ads', 'social_media', 'events', 'referral', 'organic_search',
       nan], dtype=object)

In [202]:
df['industry'].unique()




array([nan, 'retail', 'healthcare', 'education', 'manufacturing',
       'technology', 'other', 'finance'], dtype=object)

In [204]:
df['employment_status'].unique()

array(['unemployed', 'employed', nan, 'self_employed', 'student'],
      dtype=object)

In [206]:
df['location'].unique()

array(['south_america', 'australia', 'europe', 'africa', 'middle_east',
       nan, 'north_america', 'asia'], dtype=object)

In [208]:
df['converted'].unique()

array([1, 0], dtype=int64)

In [210]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [212]:
# numerical columns
df['number_of_courses_viewed'].unique()

array([1, 5, 2, 3, 0, 4, 6, 8, 7, 9], dtype=int64)

In [214]:
# missing values
df['number_of_courses_viewed'].isnull().sum()


0

In [216]:
df['annual_income'].isnull().sum()

181

In [218]:
# Fill missing income values with 0
df['annual_income'] = df['annual_income'].fillna(0.0)



In [220]:
df['annual_income'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
1457    False
1458    False
1459    False
1460    False
1461    False
Name: annual_income, Length: 1462, dtype: bool

In [222]:
df['annual_income'].isnull().sum()

0

In [224]:
# Fill missing categorical values with placeholders
df['lead_score'].isnull().sum()
df['lead_source'] = df['lead_source'].fillna("NA")
df['lead_source'].isnull().sum()

0

In [226]:
df['industry'] = df['industry'].fillna("NA")
df['industry'].isnull().sum()

0

In [228]:
df['employment_status'] = df['employment_status'].fillna("NA")
df['employment_status'].isnull().sum()

0

In [230]:
df['location'] = df['location'].fillna("NA")
df['location'].isnull().sum()

0

In [232]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [234]:


# Split dataset into train and test
data_full_train, data_test = train_test_split(df, test_size=0.2, random_state=1)

# Define numerical columns
numeric_features = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

# Correlation check
print("interaction_count vs numerical:")
data_full_train[numeric_features].corrwith(data_full_train['interaction_count']).abs().sort_values(ascending=False)

print("lead_score vs numerical:")
data_full_train[numeric_features].corrwith(data_full_train['lead_score']).abs().sort_values(ascending=False)

interaction_count vs numerical:
lead_score vs numerical:


lead_score                  1.000000
annual_income               0.031716
interaction_count           0.011290
number_of_courses_viewed    0.010711
dtype: float64

In [236]:
# Split data into train/val/test
data_full_train, data_test = train_test_split(df, test_size=0.2, random_state=42)
data_train, data_val = train_test_split(data_full_train, test_size=0.25, random_state=42)
print(len(data_train), len(data_val), len(data_test))

876 293 293


In [238]:
# Reset indices
data_train = data_train.reset_index(drop=True)
data_val = data_val.reset_index(drop=True)
data_test = data_test.reset_index(drop=True)

In [240]:
# Separate target variable
y_train = data_train['converted'].values
y_val = data_val['converted'].values
y_test = data_test['converted'].values

del data_train['converted']
del data_val['converted']
del data_test['converted']

data_full_train = data_full_train.reset_index(drop=True)
data_full_train.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [242]:
# Define categorical features
categorical_features = ['lead_source', 'employment_status', 'location', 'industry']

# Compute mutual information scores
round(mutual_info_score(data_full_train['converted'], data_full_train['industry']), 2)
round(mutual_info_score(data_full_train['converted'], data_full_train['location']), 2)
round(mutual_info_score(data_full_train['converted'], data_full_train['lead_source']), 2)
round(mutual_info_score(data_full_train['converted'], data_full_train['employment_status']), 2)


0.01

In [244]:
# One-Hot Encoding using DictVectorizer
dv = DictVectorizer(sparse=False)
train_records = data_train[categorical_features + numeric_features].to_dict(orient='records')
print(train_records[0])
X_train = dv.fit_transform(train_records)
X_train

dv.get_feature_names_out()

val_records = data_val[categorical_features + numeric_features].to_dict(orient='records')
X_val = dv.transform(val_records)

{'lead_source': 'paid_ads', 'employment_status': 'student', 'location': 'middle_east', 'industry': 'retail', 'number_of_courses_viewed': 0, 'annual_income': 58472.0, 'interaction_count': 5, 'lead_score': 0.03}


In [246]:
# Train Logistic Regression model
log_reg_model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
log_reg_model.fit(X_train, y_train)

log_reg_model.coef_.shape
log_reg_model.coef_[0].round(3)
log_reg_model.intercept_

df.columns
categorical_features
numeric_features

# Predict probabilities
y_pred = log_reg_model.predict_proba(X_val)[:, 1]
y_pred

# Convert probabilities to binary predictions (0.5 threshold)
will_convert = (y_pred >= 0.5)
will_convert


array([ True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True, False,  True, False,  True,  True,
       False, False,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True, False, False,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,

In [248]:
# Calculate accuracy
val_accuracy = (y_val == will_convert).mean()
val_accuracy
round(val_accuracy, 2)

0.7

In [250]:

def train_and_evaluate_model(train_df, val_df, y_train_values, y_val_values, feature_columns):
    """
    Train a Logistic Regression model and return accuracy score.
    """
    # Convert DataFrames to dictionaries
    dv = DictVectorizer(sparse=False)
    train_dict = train_df[feature_columns].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    
    val_dict = val_df[feature_columns].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    
    # Train model
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train_values)
    
    # Predict probabilities
    y_pred = model.predict_proba(X_val)[:, 1]
    y_pred_labels = (y_pred >= 0.5).astype(int)
    
    # Compute accuracy
    acc = accuracy_score(y_val_values, y_pred_labels)
    
    return acc


In [258]:
def feature_exclusion_test(train_df, val_df, y_train_values, y_val_values, all_features):
    """
    Test model accuracy when excluding one feature at a time.
    """
    acc_all = train_and_evaluate_model(train_df, val_df, y_train_values, y_val_values, all_features)
    print(f"All features: {acc_all:.4f}")
    
    least_useful_feature = ''
    least_useful_acc = 1
    least_useful_acc_delta = 1

    for feature in all_features:
        reduced_features = [f for f in all_features if f != feature]
        acc = train_and_evaluate_model(train_df, val_df, y_train_values, y_val_values, reduced_features)
        print(f"\nWithout '{feature}': {acc:.4f}")
        delta = abs(acc_all - acc)
        print("Current usefulness delta:", delta)
        if delta < least_useful_acc_delta:
            least_useful_acc = acc
            least_useful_feature = feature
            least_useful_acc_delta = delta

    print("\n\nSummary:")
    print("All features accuracy:", acc_all)
    print("Least useful feature:", least_useful_feature)
    print("Accuracy without it:", least_useful_acc)
    print("Accuracy delta:", least_useful_acc_delta)


len(df.columns)
numeric_features
categorical_features

input_features = categorical_features + numeric_features
input_features

feature_exclusion_test(data_train, data_val, y_train, y_val, input_features)

TypeError: 'numpy.float64' object is not callable

In [254]:
# Train and display final model
dv = DictVectorizer(sparse=False)
train_dict = data_train[categorical_features + numeric_features].to_dict(orient='records')
print(train_dict[0])
X_train = dv.fit_transform(train_dict)
print(X_train)
print(dv.get_feature_names_out())

val_dict = data_val[categorical_features + numeric_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
print(model.coef_[0].round(3))
print(model.intercept_)

y_pred = model.predict_proba(X_val)[:, 1]
print(y_pred)

accuracy_score = (y_val == will_convert).mean()
accuracy_score

{'lead_source': 'paid_ads', 'employment_status': 'student', 'location': 'middle_east', 'industry': 'retail', 'number_of_courses_viewed': 0, 'annual_income': 58472.0, 'interaction_count': 5, 'lead_score': 0.03}
[[5.8472e+04 0.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [7.1738e+04 0.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 3.0000e+00]
 [8.1973e+04 0.0000e+00 1.0000e+00 ... 1.0000e+00 0.0000e+00 3.0000e+00]
 ...
 [8.9042e+04 0.0000e+00 1.0000e+00 ... 0.0000e+00 0.0000e+00 3.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 [5.0259e+04 0.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 4.0000e+00]]
['annual_income' 'employment_status=NA' 'employment_status=employed'
 'employment_status=self_employed' 'employment_status=student'
 'employment_status=unemployed' 'industry=NA' 'industry=education'
 'industry=finance' 'industry=healthcare' 'industry=manufacturing'
 'industry=other' 'industry=retail' 'industry=technology'
 'interaction_count' '

0.6996587030716723

In [256]:
# Logistic Regression with different regularization values
def accuracy_with_regularization(train_df, val_df, feature_columns, y_train_values, y_val_values, C=1.0):
    """
    Train a Logistic Regression model with a specific regularization strength (C)
    and print the validation accuracy.
    """
    dv = DictVectorizer(sparse=False)
    train_dict = train_df[feature_columns].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    
    val_dict = val_df[feature_columns].to_dict(orient='records')
    X_val = dv.transform(val_dict)

    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train_values)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    will_convert = (y_pred >= 0.5)
    accuracy = round((y_val_values == will_convert).mean(), 3)
    
    print(f"Validation Accuracy (C={C}): {accuracy:.4f}")
    return accuracy

In [185]:
# Try different regularization strengths
for c in [0.01, 0.1, 1, 10, 100]:
    accuracy_with_regularization(data_train, data_val, input_features, y_train, y_val, C=c)

Validation Accuracy (C=0.01): 0.7000
Validation Accuracy (C=0.1): 0.7000
Validation Accuracy (C=1): 0.7000
Validation Accuracy (C=10): 0.7000
Validation Accuracy (C=100): 0.7000
